In [7]:
import win32com.client.gencache as gc
import win32com
c = win32com.client.constants

After a couple hours of trying everything I could think of, the following series of commands are the ONLY way I could figure out how to get data from the DataSocket to update. Specifically, The correct ReadMode, AccessMode must be used, and you must NOT call DataSocket.Update(). 

Other attempts resulted in complete failure to obtain data, obtain data only on 1st call, and in a couple places stalled the working thread. 

In [8]:
try:
    ds.SyncDisconnect(-1)
except NameError:
    pass

ds = gc.EnsureDispatch("CWDSLib.CWDataSocket")
ds.ReadMode = c.cwdsReadsSynchronous        
ds.SyncConnectTo("dstp://localhost/test", c.cwdsReadAutoUpdate, -1)
ds.SyncRead(-1)

False

In [19]:
if ds.Status == 2:
    ds.SyncRead(10)
    print(repr(ds.Data.Value))
else:
    print(ds.LastMessage)

(3613143241.167845, ('TempPV(C)', 'TempAIsPrimaryActual', 'TempHeatDutyActual(%)', 'TempModeActual', 'TempARaw(C)', 'TempBRaw(C)', 'InterlockHeater', 'TempAIsActive', 'TempBIsActive', 'TempA(C)', 'TempB(C)', 'TempHeatDutyUser(%)', 'TempSP(C)', 'TempModeUser', 'TempHardware', 'TempUserConfig', 'FilterOvenDutyUser(%)', 'FilterOvenSP(C)', 'Not Used 0', 'AgModeActual', 'AgPV(RPM)', 'AgMainGasActualRequest(LPM)', 'AgMainGasUser(LPM)', 'AgModeUser', 'AgSP(RPM)', 'CalLimitsPressure', 'SystemVars(XML)', 'Flattened Bag String', 'RIO Version', 'BioreactorModel', 'Cal Values', 'LEDWhiteLEDOn', 'pHAIsPrimaryActual', 'pHBRaw', 'pHARaw', 'pHCO2ActualRequest(%)', 'pHBaseDutyActual(%)', 'pHA', 'pHModeActual', 'pHActiveMode', 'pHAIsActive', 'pHBIsActive', 'pHPV', 'pHB', 'pHCO2User(%)', 'pHBaseDutyUser(%)', 'pHSP', 'pHModeUser', 'pHHardware', 'pHUserConfig', 'DOAIsPrimaryActual', 'DOBRaw(%)', 'DOARaw(%)', 'DON2FlowActualRequest(%)', 'DOO2FlowControllerRequestLimited(mLPM)', 'DOA(%)', 'DOAisActive', 'DOB

In [14]:
oldvalue = -1

In [16]:
ds.SyncRead(-1)
#ds.Update()
#ds.SyncRead(-1)
value = ds.Data.Value
print(value == oldvalue)
oldvalue = value
value

False


(('TempPV(C)',
  'TempAIsPrimaryActual',
  'TempHeatDutyActual(%)',
  'TempModeActual',
  'TempARaw(C)',
  'TempBRaw(C)',
  'InterlockHeater',
  'TempAIsActive',
  'TempBIsActive',
  'TempA(C)',
  'TempB(C)',
  'TempHeatDutyUser(%)',
  'TempSP(C)',
  'TempModeUser',
  'TempHardware',
  'TempUserConfig',
  'FilterOvenDutyUser(%)',
  'FilterOvenSP(C)',
  'Not Used 0',
  'AgModeActual',
  'AgPV(RPM)',
  'AgMainGasActualRequest(LPM)',
  'AgMainGasUser(LPM)',
  'AgModeUser',
  'AgSP(RPM)',
  'CalLimitsPressure',
  'SystemVars(XML)',
  'Flattened Bag String',
  'RIO Version',
  'BioreactorModel',
  'Cal Values',
  'LEDWhiteLEDOn',
  'pHAIsPrimaryActual',
  'pHBRaw',
  'pHARaw',
  'pHCO2ActualRequest(%)',
  'pHBaseDutyActual(%)',
  'pHA',
  'pHModeActual',
  'pHActiveMode',
  'pHAIsActive',
  'pHBIsActive',
  'pHPV',
  'pHB',
  'pHCO2User(%)',
  'pHBaseDutyUser(%)',
  'pHSP',
  'pHModeUser',
  'pHHardware',
  'pHUserConfig',
  'DOAIsPrimaryActual',
  'DOBRaw(%)',
  'DOARaw(%)',
  'DON2FlowAct

In [ ]:
import matplotlib.pyplot as plt
%matplotlib

In [ ]:
f = plt.figure()
ax = f.add_subplot(111)
line, = ax.plot(ds.Data.Value)
f.show()

In [ ]:
while True:
    ds.SyncRead(-1)
    y = ds.Data.Value
    x = list(range(len(y)))
    line.set_data(x, y)
    f.canvas.flush_events()
    f.canvas.draw()

In [ ]:
import win32com.client.gencache as gc
import win32com
c = win32com.client.constants

Just kidding, this way works too. 

http://zone.ni.com/reference/en-XX/help/375857A-01/html/t_nationalinstruments_net_readmode/
http://zone.ni.com/reference/en-XX/help/375857A-01/xml/usingdatasocketeventsaccessmodesnet/

tldr: in AutoUpdate access mode, the OnDataUpdated event is generated when data is changed. Otherwise, must use Update(). 
In ReadAsynchronous ReadMode, data is read automatically when updated from the server (aka pushed from server to client). Otherwise, muse use SyncRead(). 

TODO: Determine the difference between buffered and unbuffered reads. 

In [ ]:
class EventHandler:
    def OnStatusUpdated(self, *args):
        #print("OnStatusUpdate")
        pass
    def OnDataUpdated(self, ob):
        ob.Invoke(1, 0, pythoncom.DISPATCH_PROPERTYGET, 1)
        print("test")

ds = gc.EnsureDispatch("CWDSLib.CWDataSocket")
handler = win32com.client.WithEvents(ds, EventHandler)
ds.ReadMode = c.cwdsReadsAsynchronous        
# ds.SyncConnectTo("dstp://localhost/test", c.cwdsReadAutoUpdate, -1)
ds.ConnectTo("dstp://localhost/test", c.cwdsReadAutoUpdate)
#ds.SyncRead(-1)
oldvalue = -1

In [ ]:
#ds.SyncRead(-1)
#ds.Update()
#ds.SyncRead(-1)
value = ds.Data.Value
print(value == oldvalue)
oldvalue = value
value

In [ ]:
ds.Disconnect()

In [ ]:
dir(lastargs[0])

In [ ]:
lastarg = lastargs[0]
info = lastarg.GetTypeInfo()

In [ ]:
dir(info)

In [ ]:
info.GetNames(1)

In [ ]:
lastarg.Value

In [ ]:
import pythoncom

In [ ]:
pythoncom.DISPATCH_

In [ ]:
lastarg.Invoke(1, 0, pythoncom.DISPATCH_PROPERTYGET, 1)